In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


# define dataset
df = pd.read_csv('bodyfat_dataset.csv')
df.sample(frac = 1) # shuffle
df.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [8]:
labels = df.loc[:, ['Density', 'BodyFat']]
labels = np.array(labels)

labels.shape

(252, 2)

In [9]:
# drop the data labels
df = df.drop('Density', axis=1)
df = df.drop('BodyFat', axis=1)
df.head()

,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [10]:
# scale data
data = np.array(df)

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X = x_scaler.fit_transform(data)
Y = y_scaler.fit_transform(labels)

print(X.shape)
print(Y.shape)

(252, 13)
(252, 2)


In [11]:
X_train = X[:200]
Y_train = Y[:200]

X_test = X[-52:]
Y_test = Y[-52:]

print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

(200, 13)
(200, 2)
(52, 13)
(52, 2)


In [12]:
import nnet

model = nnet.Sequential([
    nnet.Layer((13, 10), nnet.activation.Tanh),
    nnet.Layer((10, 8), nnet.activation.Tanh),
    nnet.Layer((8, 2), nnet.activation.Sigmoid)
])

model.fit(X_train, Y_train, 23, nnet.loss.MeanSquaredError, nnet.optimizers.RMSProp(), X_val=X_test, Y_val=Y_test)

Epoch: 1 Loss: 0.0802836968697866 Validation Loss: 0.10568621572552352 
Epoch: 2 Loss: 0.054085745252542186 Validation Loss: 0.07781144089497237 
Epoch: 3 Loss: 0.031236509154294476 Validation Loss: 0.04494187093386854 
Epoch: 4 Loss: 0.02890991213183349 Validation Loss: 0.037758464654638384 
Epoch: 5 Loss: 0.027433993954072724 Validation Loss: 0.04047931618473119 
Epoch: 6 Loss: 0.02605842673557826 Validation Loss: 0.03424040307998738 
Epoch: 7 Loss: 0.025188089748269134 Validation Loss: 0.038354126479228236 
Epoch: 8 Loss: 0.023912131679502943 Validation Loss: 0.031127743060758453 
Epoch: 9 Loss: 0.02341801617757254 Validation Loss: 0.03645595876950477 
Epoch: 10 Loss: 0.02200992827455331 Validation Loss: 0.02876641768796116 
Epoch: 11 Loss: 0.021477104805962145 Validation Loss: 0.03366431549332834 
Epoch: 12 Loss: 0.02022113163197509 Validation Loss: 0.026871800703896557 
Epoch: 13 Loss: 0.019729560859975452 Validation Loss: 0.030905751657317643 
Epoch: 14 Loss: 0.018845083320857933

In [13]:
# test model accuracy using Mean Absolute Error
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)
mean_absolute_error(Y_test, y_pred.T)

0.11933460781324844

In [14]:
# actual testing
def test_prediction(index):
    y_pred = model.predict(np.array([X_test[index]]))
    y_true = np.array([Y_test[index]])
    
    print('Prediction', y_scaler.inverse_transform(y_pred.T))
    print('Actual', y_scaler.inverse_transform(y_true))

In [15]:
test_prediction(18)

Prediction [[ 1.04508678 25.38302848]]
Actual [[ 1.0433 24.5   ]]


In [16]:
test_prediction(19)

Prediction [[ 1.05643483 18.35557115]]
Actual [[ 1.0646 15.    ]]


In [53]:
test_prediction(27)

Prediction [[ 1.04361931 24.46933901]]
Actual [[ 1.0418 25.2   ]]
